In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division
from __future__ import print_function

import sys, os, gc, math
import tensorflow as tf
import numpy as np
from sklearn import linear_model
from sklearn.metrics import roc_curve, auc
from scipy.fftpack import dct,idct


sys.path.append('../')
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from models.neural_network import NeuralNetwork
from models.cnn import CNN
from models.util import *


#Seed used for choosing classes, training points, and test points.
SEED = 14
#SEED=11

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Define params of model
input_shape = (32,32,3)
num_classes = 10

In [3]:
#%%capture one
#Load model from disk
model_name = 'cifar10-resnet'
model_save_path = '../trained_models/' + model_name + '-model.json'
weights_save_path = '../trained_models/' + model_name + 'weights'
model = CNN(model_name=model_name, dataset='cifar10', transform='dct-3d',seed=SEED)
print ('Total Model Params: %d' % model.num_params)
model.load_model(model_save_path, weights_save_path) 
#epochs = 200
#model.train(epochs=epochs)
#model.save_model(model_save_path, weights_save_path)    

../models/neural_network.py:172: ComplexWarning: Casting complex values to real discards the imaginary part
  x_rec[i,:,:,0]= f_recon_r
../models/neural_network.py:173: ComplexWarning: Casting complex values to real discards the imaginary part
  x_rec[i,:,:,1]= f_recon_g
../models/neural_network.py:174: ComplexWarning: Casting complex values to real discards the imaginary part
  x_rec[i,:,:,2]= f_recon_b


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Learning rate:  0.001
Total Model Params: 273066
Loaded model from disk


In [4]:
#Model Accuracy
print ('Model Accuracy: %.5f' % (model.model.evaluate(model.val_data, model.val_labels)[1]))

10000/10000 [==============================] - 3s 316us/step
Model Accuracy: 0.83780


In [5]:
%%capture two
#Get training samples
num_train_samples = 1000
data_indices = model.gen_rand_indices(low=int(model.train_data.shape[0]/2), high=model.train_data.shape[0], seed=SEED, num_samples=num_train_samples)
train_data = model.train_data[data_indices]
train_data_labels = model.train_labels[data_indices]
train_data_labels_int = np.argmax(train_data_labels, axis=1)

**GREYBOX ATTACKS**

In [6]:
%%capture three
num_test_samples_per_class = 100
num_test_samples = num_classes*num_test_samples_per_class

#Generate test points
test_indices = model.gen_rand_indices_all_classes(y=model.test_labels, seed=SEED, num_samples=num_test_samples_per_class)

In [7]:
targets = model.test_labels[test_indices]

In [8]:
for t in targets:
    c = np.where(t==1)[0][0]
    nc = (c + 1) %10
    t[c] = 0
    t[nc] = 1


In [ ]:
%%capture four
import time
#Get Regular, JSMA, DF, and CW test points
reg_data = model.test_data[test_indices]
start_time = time.time()
#jsma_data = model.generate_perturbed_data(model.test_data[test_indices], model.test_labels[test_indices], seed=SEED, perturbation='JSMA', iterations=100)
print("--- JSMA: %s seconds ---" % (time.time() - start_time))
start_time = time.time()
cwl0_data = model.generate_perturbed_data(model.test_data[test_indices] ,seed=SEED, perturbation='CW-l0')
print("--- CW: %s seconds ---" % (time.time() - start_time))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
#%%capture five
reg_data = np.load('data/reg_data_cifar10_l0.npy')
jsma_data = np.load('data/jsma_data_cifar10_l0.npy')
cwl0_data = np.load('data/cwl0_data_cifar10_l0.npy')


In [ ]:
%%capture five
#np.save('data/reg_data_cifar10_l0', reg_data)
#np.save('data/jsma_data_cifar10_l0', jsma_data)
np.save('data/cwl0_data_cifar10_l0_v2', cwl0_data)

In [ ]:
%%capture six
#Reset tf.graph() as Cleverhans modifies the graph
tf.reset_default_graph()

#Reload the model and weights
model = CNN(model_name=model_name, dataset='cifar10',transform='dct-3d', seed=SEED)
model.load_model(model_save_path, weights_save_path)    

In [9]:
print ('Model Accuracy REG: %.5f' % (model.model.evaluate(reg_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy CW-l0: %.5f' % (model.model.evaluate(cwl0_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy JSMA: %.5f' % (model.model.evaluate(jsma_data,model.test_labels[test_indices])[1]))

1000/1000 [==============================] - 0s 173us/step
Model Accuracy REG: 0.87800
1000/1000 [==============================] - 0s 172us/step
Model Accuracy CW-l0: 0.29300
1000/1000 [==============================] - 0s 172us/step
Model Accuracy JSMA: 0.01500


In [ ]:
#Lets visualize training data
x_vis = np.random.choice(range(0,num_train_samples), 10)
print ('Training Data: ')
visualize(model.val_data[x_vis].reshape(-1,*input_shape), 10, '../figures/train-cifar10_l0')

In [ ]:
#Lets visualize samples from each attack
x_vis = np.random.choice(range(0,num_test_samples), 10)
print ('Regular: ')
visualize(reg_data[x_vis].reshape(-1,*input_shape), 10, '../figures/reg-cifar10_l0')
print ('CW-l0: ')
visualize(cwl0_data[x_vis].reshape(-1,*input_shape), 10, '../figures/cwl0-cifar10_l0')
print ('JSMA: ')
visualize(jsma_data[x_vis].reshape(-1,*input_shape), 10, '../figures/jsma-cifar10_l0')

In [10]:
%%capture seven
#Get predictions
reg_preds = model.model.predict(reg_data.reshape(-1,*input_shape))
cwl0_preds = model.model.predict(cwl0_data.reshape(-1,*input_shape))
jsma_preds = model.model.predict(jsma_data.reshape(-1,*input_shape))

#Convert preds to labels
reg_labels = preds_to_labels(reg_preds)
cwl0_labels = preds_to_labels(cwl0_preds)
jsma_labels = preds_to_labels(jsma_preds)

#Select Adversarial Points (i.e. points that lead to misclassification)
true_preds = np.argmax(model.test_labels[test_indices], axis=1)

#Check which points are actually adversarial and select those
cwl0_idx = np.where(np.argmax(cwl0_preds, axis=1) != true_preds)[0]
jsma_idx = np.where(np.argmax(jsma_preds, axis=1) != true_preds)[0]


#Filter data points to be used for similarity
cwl0_data_fil = cwl0_data[cwl0_idx]
jsma_data_fil = jsma_data[jsma_idx]


#Filter labels to be used
cwl0_labels_fil = cwl0_labels[cwl0_idx]
jsma_labels_fil = jsma_labels[jsma_idx]

In [11]:
#Get l-0 distortion 
print ('CW-l0: %.5f' % (avg_l0_dist(reg_data[cwl0_idx], cwl0_data_fil)))
print ('JSMA: %.5f' % (avg_l0_dist(reg_data[jsma_idx], jsma_data_fil)))

CW-l0: 1279.00000
JSMA: 35.00000


In [12]:
n = 32
k=75

In [13]:
clean_reg = np.zeros(reg_data.shape)
for i in range(num_test_samples):
    f_x_r = dct(reg_data[i,:,:,0].flatten(),norm='ortho').reshape(int(n),int(n))
    f_x_g = dct(reg_data[i,:,:,1].flatten(),norm='ortho').reshape(int(n),int(n))
    f_x_b = dct(reg_data[i,:,:,2].flatten(),norm='ortho').reshape(int(n),int(n))
    top_k_r = get_top_k(f_x_r,k=k)
    top_k_g = get_top_k(f_x_g,k=k)
    top_k_b = get_top_k(f_x_b,k=k)
    f_recon_r = idct(top_k_r.flatten(),norm='ortho').reshape(int(n),int(n))
    f_recon_g = idct(top_k_g.flatten(),norm='ortho').reshape(int(n),int(n))
    f_recon_b = idct(top_k_b.flatten(),norm='ortho').reshape(int(n),int(n))
  
    clean_reg[i,:,:,0]= f_recon_r
    clean_reg[i,:,:,1]= f_recon_g
    clean_reg[i,:,:,2]= f_recon_b

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  del sys.path[0]
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:15: ComplexWarning: Casting complex values to real discards the imaginary part
  from ipykernel import kernelapp as app


In [ ]:
avg=0.0
ratios = []
for i,j in enumerate(cwl0_idx):
    r = np.linalg.norm(reg_data[j]-cwl0_data_fil[i])/np.linalg.norm(reg_data[j])
    ratios.append(r)
    avg+=r
print(avg/1000, max(ratios), min(ratios))

In [ ]:
avg=0.0
ratios = []
for i,j in enumerate(jsma_idx):
    r = np.linalg.norm(reg_data[j]-jsma_data_fil[i])/np.linalg.norm(reg_data[j])
    ratios.append(r)
    avg+=r
print(avg/1000, max(ratios), min(ratios))

In [17]:
%%capture eleven
#Reconstruct CW-l0
clean_cwl0_iht = np.zeros(cwl0_data.shape)
for i in range(num_test_samples):
    x_adv_r = cwl0_data[i,:,:,0].flatten()
    x_adv_g = cwl0_data[i,:,:,1].flatten()
    x_adv_b = cwl0_data[i,:,:,2].flatten()
    t_r = len(np.where(np.abs(x_adv_r - reg_data[i,:,:,0].flatten()) > 0.0001)[0])
    t_g = len(np.where(np.abs(x_adv_g - reg_data[i,:,:,1].flatten()) > 0.0001)[0])
    t_b = len(np.where(np.abs(x_adv_b - reg_data[i,:,:,2].flatten()) > 0.0001)[0])
    x_hat_r,_ = iht(x_adv_r,35, T=100,k=75)
    x_hat_g,_ = iht(x_adv_g,35, T=100,k=75)
    x_hat_b,_ = iht(x_adv_b,35, T=100,k=75)
    clean_cwl0_iht[i,:,:,0]= idct(x_hat_r.flatten(),norm='ortho').reshape(n,n)
    clean_cwl0_iht[i,:,:,1]= idct(x_hat_g.flatten(),norm='ortho').reshape(n,n)
    clean_cwl0_iht[i,:,:,2]= idct(x_hat_b.flatten(),norm='ortho').reshape(n,n)

In [ ]:
%%capture twelve
#Reconstruct CW-l0
clean_jsma_iht = np.zeros(jsma_data.shape)
for i in range(num_test_samples):
    x_adv_r = jsma_data[i,:,:,0].flatten()
    x_adv_g = jsma_data[i,:,:,1].flatten()
    x_adv_b = jsma_data[i,:,:,2].flatten()
    t_r = len(np.where(np.abs(x_adv_r - reg_data[i,:,:,0].flatten()) > 0.0001)[0])
    t_g = len(np.where(np.abs(x_adv_g - reg_data[i,:,:,1].flatten()) > 0.0001)[0])
    t_b = len(np.where(np.abs(x_adv_b - reg_data[i,:,:,2].flatten()) > 0.0001)[0])
    print(t_r,t_g,t_b)
    x_hat_r,_ = iht(x_adv_r,t_r, T=100,k=75)
    x_hat_g,_ = iht(x_adv_g,t_g, T=100,k=75)
    x_hat_b,_ = iht(x_adv_b,t_b, T=100,k=75)
    clean_jsma_iht[i,:,:,0]= idct(x_hat_r.flatten(),norm='ortho').reshape(n,n)
    clean_jsma_iht[i,:,:,1]= idct(x_hat_g.flatten(),norm='ortho').reshape(n,n)
    clean_jsma_iht[i,:,:,2]= idct(x_hat_b.flatten(),norm='ortho').reshape(n,n)

In [18]:
np.save('data/clean_cifar10_cwl0_l0-iht', clean_cwl0_iht)

In [ ]:
%%capture thirteen
np.save('data/clean_cifar10_cwl0_l0-iht', clean_cwl0_iht)
np.save('data/clean_cifar10_jsma_l0-iht', clean_jsma_iht)

In [ ]:
visualize(clean_reg[x_vis].reshape(-1,*input_shape), 10, '../figures/reg-test-cifar10_l0')
visualize(clean_cwl0_iht[x_vis].reshape(-1,*input_shape), 10, '../figures/cwl0-clean-cifar10_l0-iht')
visualize(clean_jsma_iht[x_vis].reshape(-1,*input_shape), 10, '../figures/jsma-clean-cifar10_l0-iht')

In [19]:
print ('Model Accuracy Reconstructed CW-l0: %.5f' % (model.model.evaluate(clean_cwl0_iht,model.test_labels[test_indices])[1]))
#print ('Model Accuracy Reconstructed JSMA: %.5f' % (model.model.evaluate(clean_jsma_iht,model.test_labels[test_indices])[1]))

1000/1000 [==============================] - 0s 192us/step
Model Accuracy Reconstructed CW-l0: 0.54800


In [ ]:
D = get_matrix(n*n)
A = np.concatenate((D.T,np.identity(n*n)),axis=1)

In [ ]:
#%%capture thirteen
#Reconstruct CW-l0
clean_cwl0_socp = np.zeros(cwl0_data.shape)
for i in range(num_test_samples):
    x_adv_r = cwl0_data[i,:,:,0].flatten()
    x_adv_g = cwl0_data[i,:,:,1].flatten()
    x_adv_b = cwl0_data[i,:,:,2].flatten()
    
    x_hat_orig_r = dct(reg_data[i,:,:,0].flatten(),norm='ortho')
    x_hat_orig_g = dct(reg_data[i,:,:,1].flatten(),norm='ortho')
    x_hat_orig_b = dct(reg_data[i,:,:,2].flatten(),norm='ortho')

  
    x_top_r,x_bot_r = get_top_bot_k_vec(x_hat_orig_r,k=75)
    x_top_g,x_bot_g = get_top_bot_k_vec(x_hat_orig_g,k=75)
    x_top_b,x_bot_b = get_top_bot_k_vec(x_hat_orig_b,k=75)

    eta_r = np.linalg.norm(x_bot_r)/2.0
    eta_g = np.linalg.norm(x_bot_g)/2.0
    eta_b = np.linalg.norm(x_bot_b)/2.0

    print(eta_r,eta_g,eta_b)
    print(np.linalg.norm(x_adv_r - reg_data[i,:,:,0].flatten()),np.linalg.norm(x_adv_g - reg_data[i,:,:,1].flatten()),np.linalg.norm(x_adv_b - reg_data[i,:,:,2].flatten()) )

    x_hat_r = socp(x_adv_r,A, n=n*n*2,eta=eta_r)[:n*n]
    x_hat_g = socp(x_adv_g,A, n=n*n*2,eta=eta_g)[:n*n]
    x_hat_b = socp(x_adv_b,A, n=n*n*2,eta=eta_b)[:n*n]
    x_hat_top_r,_ = get_top_bot_k_vec(x_hat_r,k=75)
    x_hat_top_g,_ = get_top_bot_k_vec(x_hat_g,k=75)
    x_hat_top_b,_ = get_top_bot_k_vec(x_hat_b,k=75)
    clean_cwl0_socp[i,:,:,0]= idct(x_hat_top_r.flatten(),norm='ortho').reshape(n,n)
    clean_cwl0_socp[i,:,:,1]= idct(x_hat_top_g.flatten(),norm='ortho').reshape(n,n)
    clean_cwl0_socp[i,:,:,2]= idct(x_hat_top_b.flatten(),norm='ortho').reshape(n,n)

In [ ]:
%%capture thirteen
#Reconstruct CW-l0
clean_jsma_socp = np.zeros(jsma_data.shape)
for i in range(num_test_samples):
    x_adv_r = jsma_data[i,:,:,0].flatten()
    x_adv_g = jsma_data[i,:,:,1].flatten()
    x_adv_b = jsma_data[i,:,:,2].flatten()
    
    x_hat_orig_r = dct(reg_data[i,:,:,0].flatten(),norm='ortho')
    x_hat_orig_g = dct(reg_data[i,:,:,1].flatten(),norm='ortho')
    x_hat_orig_b = dct(reg_data[i,:,:,2].flatten(),norm='ortho')

  
    x_top_r,x_bot_r = get_top_bot_k_vec(x_hat_orig_r,k=75)
    x_top_g,x_bot_g = get_top_bot_k_vec(x_hat_orig_g,k=75)
    x_top_b,x_bot_b = get_top_bot_k_vec(x_hat_orig_b,k=75)

    eta_r = np.linalg.norm(x_bot_r)/2.0
    eta_g = np.linalg.norm(x_bot_g)/2.0
    eta_b = np.linalg.norm(x_bot_b)/2.0

    x_hat_r = socp(x_adv_r,A, n=n*n*2,eta=eta_r)[:n*n]
    x_hat_g = socp(x_adv_g,A, n=n*n*2,eta=eta_g)[:n*n]
    x_hat_b = socp(x_adv_b,A, n=n*n*2,eta=eta_b)[:n*n]
    x_hat_top_r,_ = get_top_bot_k_vec(x_hat_r,k=75)
    x_hat_top_g,_ = get_top_bot_k_vec(x_hat_g,k=75)
    x_hat_top_b,_ = get_top_bot_k_vec(x_hat_b,k=75)
    clean_jsma_socp[i,:,:,0]= idct(x_hat_top_r.flatten(),norm='ortho').reshape(n,n)
    clean_jsma_socp[i,:,:,1]= idct(x_hat_top_g.flatten(),norm='ortho').reshape(n,n)
    clean_jsma_socp[i,:,:,2]= idct(x_hat_top_b.flatten(),norm='ortho').reshape(n,n)